In [ ]:
using DrugResponseModel

# import G1, G2, and population data
conc_, pop_l, g2_l, g1_l, g2_0_l, g1_0_l = setup_data("lapatinib");
conc, pop_g, g2_g, g1_g, g2_0_g, g1_0_g = setup_data("doxorubicin");
conc_l = append!([0.1], conc_[2:end]);
conc_g = append!([0.1], conc[2:end]);

In [ ]:
# Lapatinib
ec50lowL = 50.0
ec50highL = 500.0
ec50guessL = 100.0
guessL = [ec50guessL, 1.0, 0.00682578, 0.00958013, 0.00503788, 0.0933964, 15.0, 15.0, 20.0, 20.94827, ec50guessL, 0.00294624, ec50guessL, 0.0422821]

# doxorubicin
ec50lowg = 20.0
ec50highg = 500.0
ec50guessg = 80.0
guessg = [ec50guessg, 1.0, 0.00682578, 0.00958013, 0.00503788, 0.0933964, 20.0, 20.0, 15.0, 15.94827, ec50guessg, 0.00294624, ec50guessg, 0.0422821]

# max num of steps
num_steps=10000;

In [ ]:
# best_fitL, paramsL = optimize_hill(guessL, conc_l, g1_l, g2_l, g1_0_l, g2_0_l, num_steps, ec50lowL, ec50highL)
best_fitG, paramsG = optimize_hill(guessg, conc_g, g1_g, g2_g, g1_0_g, g2_0_g, num_steps, ec50lowg, ec50highg)


In [ ]:
# parsL=getDDEparams(paramsL, conc_l)
# parsG=getDDEparams(paramsG, conc_g)
# using CSV, DataFrames
# CSV.write("parsD.csv",  DataFrame(parsG), writeheader=false)
parsD = CSV.read("parsD.csv", header=false)
parsL = CSV.read("parsL.csv", header=false)
parsG = convert(Matrix, parsD)
parsL = convert(Matrix, parsL);

In [ ]:
# plot doxorubicin
plot_all(parsG, pop_g, g2_g, g1_g, g2_0_g, g1_0_g);
# savefig("dox.png")

In [ ]:
# plot hill for doxorubicin
plot_parameters(conc_g, parsG);

In [ ]:
using Plots; pyplot();
function plot3D(parsL, parsG, j::Int, title::String)

    param1 = ParamForBliss(parsL)
    param2 = ParamForBliss(parsG)
    combin = BlissCombination(parsL, parsG)

    plot(param1[j,:],param2[j,:],combin[:,:,j],st=:surface,camera=(-40,20), xlabel="lapatinib", ylabel="doxorubicin", zlabel="combination", title=title)
    plot!(size = (800, 600), dpi=150)
end
param1 = ParamForBliss(parsL)
param2 = ParamForBliss(parsG)
combin = BlissCombination(parsL, parsG);


In [ ]:
plot3D(parsL, parsG, 1, "doxorubicin-lapatinib, alpha");
# savefig("doxlapalpha.png")

In [ ]:
plot3D(parsL, parsG, 2, "doxorubicin-lapatinib, beta");
# savefig("doxlapbeta.png")

In [ ]:
plot3D(parsL, parsG, 3, "doxorubicin-lapatinib, gamma");
# savefig("doxlapgamma.png")

In [ ]:
ddeparam = DDEcombinationParam(combin, parsL, parsG);

In [ ]:
# pyplot();
using Measures

function plotUnitCombin(params::Array, i::Int, title::String, bool::Any, g2::Matrix, g1::Matrix, g2_0::Array, g1_0::Array, concL, ConcGem)
    """ Given estimated parameters for each trial, 
    solve the DDE model plot the predicted curve 
    for # of cells in G1, G2, or total, 
    along with their corresponding real data,
    for a longer time which is 2 times of the 
    original time (~195 hours)
    """
    n_times = range(0.0; stop = 200.0, length = 400)
    alg, n_prob, _ = ddesolve(collect(n_times), g1, g2, g1_0, g2_0, params, i)

    solution = solve(n_prob, alg; constrained=true)

    plot(n_times, solution(n_times, idxs=1).u, label = "G1", xlabel = "time [hours]", ylabel = "number of cells", lw=2.0, alpha = 0.6, color =:green)
    plot!(n_times, solution(n_times, idxs=2).u, label = "G2", legend=bool, legendfontsize=7, fg_legend = :transparent, lw=2.0, alpha = 0.6, color=:sienna)
    plot!(n_times, (solution(n_times, idxs=2).u + solution(n_times, idxs=1).u), label = "total", dpi = 250, lw=2.0, alpha = 0.6, color=:hotpink, margin = 20mm)
    plot!(annotation=(100.5,125.0, text("$concL nM lapat. & $ConcGem nM Dox.", 10)))
end


function plot4combin(ddeparam, g2_l::Matrix, g1_l::Matrix, g2_0_l::Array, g1_0_l::Array, i::Int, conc_l ,conc_g)
    """ here we plot 8 combinations of lapatinib i and all the doxorubicin """ 
    concLap = conc_l[i]
    p1 = plotUnitCombin(ddeparam[i,1,:], 1, "", true, g2_l, g1_l, g2_0_l, g1_0_l, concLap, conc_g[1])
    p2 = plotUnitCombin(ddeparam[i,2,:], 1, "", false, g2_l, g1_l, g2_0_l, g1_0_l, concLap, conc_g[2])
    p3 = plotUnitCombin(ddeparam[i,3,:], 1, "", false, g2_l, g1_l, g2_0_l, g1_0_l, concLap, conc_g[3])
    p4 = plotUnitCombin(ddeparam[i,4,:], 1, "", false, g2_l, g1_l, g2_0_l, g1_0_l, concLap, conc_g[4])
    p5 = plotUnitCombin(ddeparam[i,5,:], 1, "", false, g2_l, g1_l, g2_0_l, g1_0_l, concLap, conc_g[5])
    p6 = plotUnitCombin(ddeparam[i,6,:], 1, "", false, g2_l, g1_l, g2_0_l, g1_0_l, concLap, conc_g[6])
    p7 = plotUnitCombin(ddeparam[i,7,:], 1, "", false, g2_l, g1_l, g2_0_l, g1_0_l, concLap, conc_g[7])
    p8 = plotUnitCombin(ddeparam[i,8,:], 1, "", false, g2_l, g1_l, g2_0_l, g1_0_l, concLap, conc_g[8])
    plot(p1, p2, p3, p4, p5, p6, p7, p8, layout=(2,4))
    plot!(size = (1600, 600), dpi = 250)
    ylims!((0.0, 120.0))
end

In [ ]:
plot4combin(ddeparam, g2_l, g1_l, g2_0_l, g1_0_l, 1, conc_l,conc_g)

In [ ]:
plot4combin(ddeparam, g2_l, g1_l, g2_0_l, g1_0_l, 2, conc_l,conc_g)

In [ ]:
plot4combin(ddeparam, g2_l, g1_l, g2_0_l, g1_0_l, 3, conc_l,conc_g)

In [ ]:
plot4combin(ddeparam, g2_l, g1_l, g2_0_l, g1_0_l, 4, conc_l,conc_g)

In [ ]:
plot4combin(ddeparam, g2_l, g1_l, g2_0_l, g1_0_l, 5, conc_l,conc_g)

In [ ]:
plot4combin(ddeparam, g2_l, g1_l, g2_0_l, g1_0_l, 6, conc_l,conc_g)

In [ ]:
plot4combin(ddeparam, g2_l, g1_l, g2_0_l, g1_0_l, 7, conc_l,conc_g)

In [ ]:
plot4combin(ddeparam, g2_l, g1_l, g2_0_l, g1_0_l, 8, conc_l,conc_g)